In [274]:
import glob, os, shutil
import pandas as pd
import numpy as np

In [275]:
def makedir(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [276]:
root = '/home/user1/ariel/fed_learn/large_vlm_distillation_ood/resnet18_classification_on_s_cars_dataset/s_cars_orig/'
dest = '/home/user1/ariel/fed_learn/large_vlm_distillation_ood/resnet18_classification_on_s_cars_dataset/s_cars_ood/'

In [277]:
makedir(dest)

In [278]:
makedir('s_cars_ood/train/'),makedir('s_cars_ood/test_ood/'),makedir('s_cars_ood/test_ind/')

(None, None, None)

In [279]:
train_folds, test_folds = os.listdir(root+'train/'), os.listdir(root+'test/')
for fold in train_folds:
    makedir(dest+'train/'+fold)
    makedir(dest+'test_ind/'+fold)

In [280]:
new_test_index = sum([len(os.listdir(root+'train/'+fold)) for fold in os.listdir(root+'train/')])
for fold in os.listdir(root+'test/'):
   for file in glob.glob(os.path.join(root+'test/'+fold, '*.jpg')):
       new_file_num = str(int(file.split('/')[-1].split('.')[0])+new_test_index)
       #print(root+'train/'+fold+'/'+new_file_num+'.jpg')
       shutil.copy(file, dest+'train/'+fold+'/'+new_file_num+'.jpg')
for fold in os.listdir(root+'train/'):
   for file in glob.glob(os.path.join(root+'train/'+fold, '*.jpg')):
       file_name = file.split('/')[-1]
       shutil.copy(file, dest+'train/'+fold+'/'+file_name)

In [281]:
df_train, df_test = pd.read_csv(root+'anno_train.csv',header=None), pd.read_csv(root+'anno_test.csv',header=None)
df_names = pd.read_csv(root+'names.csv', header=None)

In [282]:
def add_value_to_filename(filename, value):
    # Extract numeric part and convert to integer
    num = int(filename.split('.')[0])
    # Add the desired value
    new_num = num + value
    # Format back to the original format with leading zeros
    new_filename = f'{new_num:05d}.jpg'
    return new_filename

In [283]:
df_test[0] = df_test[0].apply(add_value_to_filename, value=new_test_index)
df_all = pd.concat([df_train,df_test], axis=0)
df_test_ood = pd.DataFrame()

In [284]:
num_train_fold = len(os.listdir(root+'train/'))
from numpy.random import default_rng
rng = default_rng()
choices = rng.choice(num_train_fold, size=10, replace=False)
# add mew classes to test
print(f'shape train before{df_all.shape} test_ood before {df_test_ood.shape}')
for choice in choices:
    condition = df_all[5]==choice
    rows_to_move = df_all[condition]
    df_test_ood = pd.concat([df_test_ood, rows_to_move], ignore_index=True)
    df_all = df_all[~condition]
del df_train
df_train = df_all
print(f'shape train after{df_train.shape}, test_ood after {df_test_ood.shape}')
selected_folds = [train_folds[i] for i in choices]
#after moved to test, I would like to move indicated classes to test df using the index I already have

shape train before(16185, 6) test_ood before (0, 0)
shape train after(15426, 6), test_ood after (759, 6)


In [285]:
for fold in selected_folds:
    shutil.move(dest+'train/'+fold, dest+'test_ood/')
df_test_ood.to_csv(dest+'test_ood.csv',header=None)
print(df_train.head())

           0    1    2     3     4    5
0  00001.jpg   39  116   569   375   14
1  00002.jpg   36  116   868   587    3
2  00003.jpg   85  109   601   381   91
3  00004.jpg  621  393  1484  1096  134
4  00005.jpg   14   36   133    99  106


## At this point start create Ind test set

In [286]:
df_test_ind = pd.DataFrame()
Num_files = 10
print(f'train shape before{df_train.shape}')
for fold in os.listdir(dest+'train/'):
    file_list = os.listdir(dest+'train/'+fold)
    sampled = random.sample(file_list, Num_files)
    for sample in sampled:
        condition = df_train[0]==sample
        rows_to_move = df_train[condition]
        df_test_ind = pd.concat([df_test_ind, rows_to_move], ignore_index=True)
        df_train = df_train[~condition]
        src, dst = f'{dest}train/{fold}/{sample}', f'{dest}test_ind/{fold}/{sample}'
        shutil.move(src,dst)
print(f'train shape after{df_train.shape}')

train shape before(15426, 6)
train shape after(13862, 6)


In [287]:
df_train.to_csv(dest+'anno_train.csv', index=None, header=None)
df_test_ind.to_csv(dest+'anno_test_ind.csv', index=None, header=None)